In [ ]:
# My Learning Box - Google Cloud - Colab
# Copyright 2023 Boris-Wilfried
#
# This Google Colab notebook is part of my "Learning Box" github repository, created for my learning purposes. 
# It is not designed for production use. 
# This notebook may serve as an inspiration or a starting point for your own explorations.
# For any queries, suggestions, or contributions, feel free to reach out to me on GitHub: https://github.com/bwnyasse

<a href="https://colab.research.google.com/github/bwnyasse/learning-box/blob/main/google-cloud/colab/langchain_bigquery/langchain_bigquery_chat_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using LangChain and GPT to chat with BigQuery data**

---

In this Colab, I want to demonstrate how to use LangChain and OpenAI GPT for querying data from Google BigQuery.

## 📒 Before I begin.








#### Setup Google Cloud Project.

1.   Create a Cloud Platform project if you do not already have one.
2.   Enable billing for the project.
3.   Enable BigQuery APIs for the project.

### Authentication and service account.

1. Create a service account if you do not already have one, with the following Bigquery Roles:

  *  BigQuery User
  *  BigQuery Data Viewer
  *  BigQuery Job User


2. Download the service account key json and put it `/content/sa-keys/sa-langchain-test-over-bigquery.json`

## 🐍 Getting Started

### Installation of required libraries.

In [ ]:
!pip install langchain openai chromadb tiktoken tabulate sqlalchemy sqlalchemy-bigquery google-cloud-bigquery &> /dev/null

### Importing libraries.

In [4]:
from google.colab import userdata
from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

## 🔐 Authenticate

### BigQuery Service Account : Option 1

In [5]:
# Option 1: For Personal Use (with Google Drive)
# Uncomment the following lines if you have stored your service account key in Google Drive.
# This is how I, the author, personally use it for demonstration purposes.
# from google.colab import drive
# drive.mount('/content/gdrive')
# service_account_file = "/content/gdrive/MyDrive/Colab Notebooks/sa-keys/sa-langchain-test-over-bigquery.json"

### BigQuery Service Account : Option 2

In [ ]:
# Option 2: General User Setup
# Uncomment and use the following line if you have placed your service account key in the /content directory of this Colab notebook.
# This is the recommended way for general users who are following the instructions from the documentation.
# service_account_file = "/content/sa-keys/sa-langchain-test-over-bigquery.json"

### OpenAI API Key

In order to interact with OpenAI's GPT models, you need to have an OpenAI API key. Here's how you can set it up:

1. **Generate an OpenAI API Key**: If you don't have one, you need to create an API key from your OpenAI account. Visit the OpenAI website, log in to your account, and navigate to the API section to generate a new key.

2. **Storing the OpenAI API Key**: For the purpose of this Colab notebook, store your OpenAI API key in a secure location. If you are using Google Colab's secret section for sensitive data, you can add your key there. Alternatively, you can store it in an environment variable or a secure file within the Colab file system.

In [8]:
from google.colab import auth
auth.authenticate_user()

# Use the following line to access your secret key (replace 'your_secret_key_name' with the actual name of your key)
# openai_key = 'your_secret_key'  # Replace with the name of your key in the secret section
openai_key = userdata.get('openAiApiKey')
os.environ["OPENAI_API_KEY"] = openai_key

## 💻 SQL Achemy Configuration

In [6]:
#@markdown Which dataset do I want to use ?

# Configuration
project = "learning-box-369917"
dataset = "langchain_test_churn_table" #@param {type:"string"}
sqlalchemy_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'

## 🔗 Initialize LangChain

Set up the LangChain with the specified configurations to prepare for executing queries.


In [9]:
db = SQLDatabase.from_uri(sqlalchemy_url)
llm = OpenAI(temperature=0, model="text-davinci-003")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    top_k=1000,
)

## 💬 Chat : Execute and Display Queries

Execute sample queries using the LangChain agent and display the responses.


### Utility to parse output into json format

In [ ]:
import json
import re

def remove_ansi_escape_codes(text):
    # ANSI escape code regex
    ansi_escape = re.compile(r'\x1B[@-_][0-?]*[ -/]*[@-~]')
    return ansi_escape.sub('', text)

def parse_response_to_json(text):
    # Initialize the structure
    response_json = {
        "ai": [],
        "final_answer": ""
    }

    current_ai_entry = {"thought": "", "action": "", "action_input": ""}
    lines = text.split('\n')
    for line in lines:
        cleaned_line = remove_ansi_escape_codes(line)

        thought_match = re.match(r'^Thought: (.+)', cleaned_line)
        action_match = re.match(r'^Action: (.+)', cleaned_line)
        action_input_match = re.match(r'^Action Input: (.+)', cleaned_line)
        final_answer_match = re.match(r'^Final Answer: (.+)', cleaned_line)

        if thought_match:
            if current_ai_entry["thought"]:  # If there's already a thought in the current entry, append it and start a new one
                response_json["ai"].append(current_ai_entry)
                current_ai_entry = {"thought": "", "action": "", "action_input": ""}

            current_ai_entry["thought"] = thought_match.group(1)

        if action_match:
            current_ai_entry["action"] = action_match.group(1)

        if action_input_match:
            current_ai_entry["action_input"] = action_input_match.group(1)

        if final_answer_match:
            response_json["final_answer"] = final_answer_match.group(1)

    # Append the last AI entry if it's not empty
    if current_ai_entry["thought"] or current_ai_entry["action"] or current_ai_entry["action_input"]:
        response_json["ai"].append(current_ai_entry)

    return json.dumps(response_json, indent=2)

import sys
import io
from contextlib import redirect_stdout

def capture_console_output(func, *args, **kwargs):
    f = io.StringIO()
    with redirect_stdout(f):
        func(*args, **kwargs)
    return f.getvalue()

### Interacting with **Churn Table** ( Sample questions )


1. **Customer Demographics:**

"What is the distribution of churned customers by gender and age group (SeniorCitizen)?"

2. **Service Usage Patterns:**

"Which internet service types have the highest churn rates among customers?"

3. **Customer Loyalty and Tenure:**

"What is the average tenure of customers who churn compared to those who stay?"

4. **Churn Analysis:**

"What are the common characteristics of customers who churn within their first year?"

5. **Billing and Payment Trends:**

"How does the average monthly charge differ between customers who churn and those who don’t?"

6. **Contract Preferences:**

"What percentage of customers on a month-to-month contract churn compared to those on one-year or two-year contracts?"

7. **Customer Satisfaction Indicators:**

"Is there a correlation between the use of tech support services and customer churn rates?"

8. **Segmentation and Personalized Marketing:**

"Can we identify distinct customer segments based on service usage patterns and churn rates?"

9. **Predictive Analysis:**

"Which factors are most predictive of churn in our customer base?"

### Interacting with **Amazon Canada Products 2023** ( Sample questions )

I have pulled the data from a public dataset available in [kaddle](https://www.kaggle.com/datasets/asaniczka/amazon-canada-products-2023-2-1m-products)

1. **Product Popularity and Performance:**

"Which products (identified by asin) have the highest number of reviews (reviews) and what are their average star ratings (stars)?"

2. **Best Sellers and Categories Analysis:**

"In which categories (categoryName) do the most bestseller products (isBestSeller = True) fall, and what is their average price (price)?"

3. **Pricing Strategy Insights:**

"For products with a high star rating (stars > 4.0), how does their selling price (price) compare to the list price (listPrice)?"

4. **Recent Purchase Trends:**

"What are the top five products that have been bought most frequently in the last month (boughtInLastMonth), and what are their categories (categoryName)?"

5. **Correlation Between Price and Popularity:**

"Is there a correlation between the price (price) of a product and its popularity in terms of reviews (reviews) or being a bestseller (isBestSeller)?"

In [ ]:
#@markdown Please fill in the value below with your request.

# Please fill in these values.
request = "What is the distribution of churned customers by gender and age group (SeniorCitizen)?" #@param {type:"string"}

# Quick input validations.
assert request, "⚠️ Please provide a request"

response_text = capture_console_output(agent_executor.run, {request})
json_output = parse_response_to_json(response_text)

print(response_text)

print(json_output)